In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import KFold
from collections import Counter
from scipy.stats import pearsonr

## Load Data

In [14]:
#creates a dataframe from a file
df = pd.read_csv("Postures.csv")

#info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78096 entries, 0 to 78095
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Class   78096 non-null  int64  
 1   User    78096 non-null  int64  
 2   X0      78096 non-null  float64
 3   Y0      78096 non-null  float64
 4   Z0      78096 non-null  float64
 5   X1      78096 non-null  float64
 6   Y1      78096 non-null  float64
 7   Z1      78096 non-null  float64
 8   X2      78096 non-null  float64
 9   Y2      78096 non-null  float64
 10  Z2      78096 non-null  float64
 11  X3      78096 non-null  object 
 12  Y3      78096 non-null  object 
 13  Z3      78096 non-null  object 
 14  X4      78096 non-null  object 
 15  Y4      78096 non-null  object 
 16  Z4      78096 non-null  object 
 17  X5      78096 non-null  object 
 18  Y5      78096 non-null  object 
 19  Z5      78096 non-null  object 
 20  X6      78096 non-null  object 
 21  Y6      78096 non-null  object 
 22

In [15]:
print(df)

       Class  User         X0          Y0         Z0         X1          Y1  \
0          0     0   0.000000    0.000000   0.000000   0.000000    0.000000   
1          1     0  54.263880   71.466776 -64.807709  76.895635   42.462500   
2          1     0  56.527558   72.266609 -61.935252  39.135978   82.538530   
3          1     0  55.849928   72.469064 -62.562788  37.988804   82.631347   
4          1     0  55.329647   71.707275 -63.688956  36.561863   81.868749   
...      ...   ...        ...         ...        ...        ...         ...   
78091      5    14  54.251127  129.177414 -44.252511  27.720784  107.810661   
78092      5    14  54.334883  129.253842 -44.016320  27.767911  107.914808   
78093      5    14  54.151540  129.269502 -44.173273  27.725978  108.034006   
78094      5    14  27.915311  108.007390  10.814957  -0.910435  122.464093   
78095      5    14  27.898705  108.092877  11.107857 -30.031402   77.740235   

              Z1         X2          Y2  ...       

In [16]:
#ignoring the first row of the dataset
df = df.values[1:]
df

array([[1, 0, 54.2638799540698, ..., '?', '?', '?'],
       [1, 0, 56.5275584513992, ..., '?', '?', '?'],
       [1, 0, 55.8499275533856, ..., '?', '?', '?'],
       ...,
       [5, 14, 54.1515399011587, ..., '?', '?', '?'],
       [5, 14, 27.9153113368356, ..., '?', '?', '?'],
       [5, 14, 27.8987054186744, ..., '?', '?', '?']], dtype=object)

In [ ]:
# variable to classify is the firts column Class, with values ranging between 1 and 5

#the user column has values ranging between 0 and 14 skipping the 3

# ? values see what to do with those, eliminating them does seem like an option , replace them with another value maybe (Nan) , imputating with other value (mean)